In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# dimensions of our images.
img_width, img_height = 128, 128

train_data_dir = './Open_I_abd_vs_CXRs/TRAIN' # location of training data
validation_data_dir = './Open_I_abd_vs_CXRs/VAL' # location of validation data

# number of samples used for determining the samples_per_epoch
nb_train_samples = 65
nb_validation_samples = 10
epochs = 10
batch_size = 5  

train_datagen = ImageDataGenerator(
    rescale = 1./255,            # normalize pixel values to [0,1]
    shear_range = 0.2,
    zoom_range = 0.2,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True)
val_datagen = ImageDataGenerator(
    rescale = 1./255)       # normalize pixel values to [0,1]

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'binary')

validation_generator = train_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'binary')

In [ ]:
base_model = applications.inception_v3.InceptionV3(weights = 'imagenet', include_top = False, input_shape = (img_width, img_height, 3))

In [ ]:
model_top = Sequential()
model_top.add(GlobalAveragePooling2D(input_shape = base_model.output_shape[1:], data_format = None)),
model_top.add(Dense(256, activation = 'relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1, activation = 'sigmoid'))

model = Model(inputs = base_model.input, outputs = model_top(base_model.output))

model.compile(optimizer = Adam(lr = 0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0),
    loss = 'binary_crossentropy',
    metrics = ['accuracy'])

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)